#Quantum Computing

Quantum computing is a research area that extends the set of physical laws classical computers operate on by accessing quantum aspects of the physical world, opening up new ways of processing information.


# Classical Machine Learning

In [4]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [8]:
y_pred = reg.predict(X_test)

In [9]:
reg.score(X_test, y_test)

0.4526027629719195

In [10]:
!pip install pennylane

In [11]:
!pip install tensorflow==2.10

#QML

In [12]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA  # For dimensionality reduction

In [13]:
# Load the diabetes dataset
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Optional: Dimensionality reduction
pca = PCA(n_components=4)  # Reduce to 4 principal components
X = pca.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the quantum device
n_qubits = X.shape[1]  # Use the reduced number of features as the number of qubits
n_layers = 1  # Use a simpler circuit with fewer layers
dev = qml.device('default.qubit', wires=n_qubits)

# Define the quantum circuit
@qml.qnode(dev, interface='tf')
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))  # Use a simpler template
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Initialize the weights
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.KerasLayer(quantum_circuit, weight_shapes, output_dim=n_qubits)

In [14]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(n_qubits,)),
    qlayer,
    tf.keras.layers.Dense(1)  # Linear output for regression
])

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse')

# Train the model in smaller batches
model.fit(X_train, y_train, epochs=10, batch_size=8, verbose=1)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

Epoch 1/10
45/45 [==============================] - 6s 141ms/step - loss: 29132.7188
Epoch 2/10
45/45 [==============================] - 5s 116ms/step - loss: 28393.9414
Epoch 3/10
45/45 [==============================] - 6s 132ms/step - loss: 27709.9434
Epoch 4/10
45/45 [==============================] - 5s 119ms/step - loss: 27053.7988
Epoch 5/10
45/45 [==============================] - 5s 117ms/step - loss: 26426.6680
Epoch 6/10
45/45 [==============================] - 6s 135ms/step - loss: 25808.2344
Epoch 7/10
45/45 [==============================] - 5s 116ms/step - loss: 25208.1504
Epoch 8/10
45/45 [==============================] - 6s 137ms/step - loss: 24628.7969
Epoch 9/10
45/45 [==============================] - 5s 113ms/step - loss: 24053.7441
Epoch 10/10
3/3 [==============================] - 0s 66ms/step


In [16]:

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 20416.958231238765
R^2 Score: -2.853600123279819
